In [1]:
import os
import locale
locale.getpreferredencoding = lambda: "UTF-8"

## **Install library**

In [2]:
!pip install -q bitsandbytes accelerate langchain transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 13.1 MB/s eta 0:00:00


In [3]:
!pip install -q sentence-transformers langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.9 MB/s eta 0:00:00


In [4]:
!pip install -q faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 74.8 MB/s eta 0:00:00


In [5]:
!pip install -q pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 11.1 MB/s eta 0:00:00


In [6]:
!pip install -q ctransformers pyvi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 107.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.6 MB/s eta 0:00:00


In [7]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain import HuggingFaceHub
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import PyPDFLoader, DirectoryLoader


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## **Define model name and vectorstores**

In [36]:
model_name = "vinai/PhoGPT-4B-Chat"
embedding_name = "dangvantuan/vietnamese-embedding"
vectore_stores = "./vectorstores"

data_path = "/content/drive/MyDrive/Data/QDQC_TDTU"

## **Load pdf file**

In [9]:
loader = PyPDFLoader("/content/drive/MyDrive/Data/SVTDTU.pdf")
documents = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter


#Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=250)
chunks = text_splitter.split_documents(documents)

In [10]:
# Set your HuggingFace API token
from google.colab import userdata

os.environ["HUGGINGFACE_API_KEY"] = userdata.get('HUGGINGFACE_API_KEY')

## **Load embeddings and llm**

In [11]:
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoConfig,BitsAndBytesConfig
import torch

def load_hf_llm(model_id, temperature=0.1):
    # Initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        model_id,
        trust_remote_code=True,
        device_map="auto"
    )

    # Initialize model
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        load_in_4bit=True,
        trust_remote_code=True,
        torch_dtype=torch.float16,  # Use float16 for memory efficiency
        device_map="auto"  # Automatically choose best device (CPU/GPU),
    )

    # Create pipeline
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_length=1024,
        temperature=temperature,
        top_p=0.95,
        top_k=50,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens = 512,

    )

    # Create LangChain HF pipeline
    llm = HuggingFacePipeline(pipeline=pipe)

    return llm

In [37]:
llm = load_hf_llm(model_name)

tokenizer_config.json:   0%|          | 0.00/850 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/844k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

configuration_mpt.py:   0%|          | 0.00/16.4k [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/vinai/PhoGPT-4B-Chat:
- custom_embedding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


adapt_tokenizer.py:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vinai/PhoGPT-4B-Chat:
- adapt_tokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/vinai/PhoGPT-4B-Chat:
- modeling_mpt.py
- meta_init_context.py
- hf_prefixlm_converter.py
- param_init_fns.py
- custom_embedding.py
- adapt_tokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin:   0%|          | 0.00/7.38G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/91.0 [00:00<?, ?B/s]

## **Create vectorstores**

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize the embedding model
embeddings = HuggingFaceEmbeddings(
    model_name="bkai-foundation-models/vietnamese-bi-encoder",
    model_kwargs={'device': 'cuda'}, # Use GPU if available
    encode_kwargs={'normalize_embeddings': True} # Normalize the embeddings
)

<ipython-input-13-3bc340a2a19b>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

In [14]:
vectorstores = FAISS.from_documents(chunks, embedding=embeddings)

In [15]:
querry = "5 ĐẶC ĐIỂM NHẬN DIỆN SINH VIÊN TRƯỜNG"
relevant_docs = vectorstores.similarity_search(querry, k=3)
relevant_docs

[Document(metadata={'source': '/content/drive/MyDrive/Data/SVTDTU.pdf', 'page': 0}, page_content='Page 1: \nBẢNG MÔ TẢ 5 ĐẶC ĐIỂM NHẬN DIỆN SINH VIÊN TRƯỜNG  \nĐẠI HỌC TÔN ĐỨC THẮNG GẮN VỚI TRIẾT LÝ GIÁO DỤC KHAI PHÓNG \n1. Kỷ luật: \n- Tôn trọng, tuân thủ các quy định của Pháp luật; \n- Chấp hành tốt nội quy, quy định của Nhà trường và địa phương nơi cư trú; \n- Sống có kỷ luật; học tập và làm việc có kế hoạch, khoa học và chuẩn mực; \n- Chịu trách nhiệm cá nhân đối với quyết định và hành vi của bản thân; \n- Thể hiện khả năng tự đánh giá, tự phê bình và điều chỉnh hành vi. \n2. Lễ phép: \n- Chào hỏi lịch sự; thể hiện thái độ chân thành, tôn trọng trong giao tiếp với thầy  \ncô, bạn bè và mọi người xung quanh; xây dựng môi trường học tập tích cực, hòa nhã; \n- Tôn trọng người khác; tôn trọng sự khác biệt về văn hóa dựa trên chuẩn mực  \nđạo đức, quy định pháp luật và quy định của Nhà trường; \n-  \nỨng xử văn minh, phù hợp với chuẩn mực đạo đức và quy định của Nhà  \ntrường; có năng l

## **Create prompt and chains to query into vectorstore**

In [38]:
from langchain import PromptTemplate

prompt_template = """Sử dụng thông tin sau đây để trả lời câu hỏi. Nếu bạn không biết câu trả lời, chỉ cần nói rằng bạn không biết, đừng cố tạo ra câu trả lời.

Thông tin: {context}

Câu hỏi: {question}

Trả lời hữu ích:"""

prompt_qa = PromptTemplate(template=prompt_template, input_variables=["context","question"])

In [39]:
from langchain.chains import RetrievalQA
retriever = vectorstores.as_retriever(search_kwargs={"k": 3})
# Create the QA chain

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstores.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": prompt_qa,
    }
)

## **Testing with query**

In [ ]:
query = "Đặc điểm đầu tiên trong 5 đặc điểm nhận dạng sinh viên Tôn ĐỨc Thắng"

response = qa_chain.invoke({"query": query})


In [ ]:
response

In [41]:
from enum import Enum
from datetime import datetime
import re
from typing import Tuple, Optional, List

class QueryType(Enum):
    CONVERSATIONAL = "conversational"
    KNOWLEDGE_BASED = "knowledge_based"
    REAL_TIME = "real_time"
    FUNCTION_CALL = "function_call"

class QueryRouter:
    def __init__(self):
        # Patterns for conversational queries
        self.conversational_patterns = [
            r'^hi\b|^hello\b|^hey\b',
            r'how are you',
            r'what\'s up',
            r'nice to meet you',
            r'good morning|good afternoon|good evening',
        ]

        # Patterns for real-time queries
        self.real_time_patterns = [
            r'what time|current time',
            r'weather in|temperature in',
            r'date today|current date',
        ]

        # Common function call indicators
        self.function_call_patterns = [
            r'calculate|compute',
            r'convert',
            r'translate',
        ]

    def classify_query(self, query: str) -> Tuple[QueryType, float]:
        query = query.lower().strip()

        # Check conversational patterns
        for pattern in self.conversational_patterns:
            if re.search(pattern, query):
                return QueryType.CONVERSATIONAL, 0.9

        # Check real-time patterns
        for pattern in self.real_time_patterns:
            if re.search(pattern, query):
                return QueryType.REAL_TIME, 0.9

        # Check function call patterns
        for pattern in self.function_call_patterns:
            if re.search(pattern, query):
                return QueryType.FUNCTION_CALL, 0.9

        # Default to knowledge-based if no other patterns match
        return QueryType.KNOWLEDGE_BASED, 0.7

    def route_query(self, query: str) -> dict:
        query_type, confidence = self.classify_query(query)

        response = {
            "query": query,
            "type": query_type.value,
            "confidence": confidence,
            "handler": None
        }

        if query_type == QueryType.CONVERSATIONAL:
            response["handler"] = "direct_llm"

        elif query_type == QueryType.REAL_TIME:
            response["handler"] = "function_handler"

        elif query_type == QueryType.FUNCTION_CALL:
            response["handler"] = "function_handler"

        else:  # KNOWLEDGE_BASED
            response["handler"] = "rag_system"

        return response

In [42]:
router = QueryRouter()

# Test queries
test_queries = [
    "Hello, how are you?",
    "What is the capital of France?",
    "What time is it now?",
    "What's the weather in Vietnam?",
    "Calculate the square root of 16",
    "Tell me about quantum physics",
]

for query in test_queries:
    result = router.route_query(query)
    print(f"\nQuery: {query}")
    print(f"Classification: {result}")



Query: Hello, how are you?
Classification: {'query': 'Hello, how are you?', 'type': 'conversational', 'confidence': 0.9, 'handler': 'direct_llm'}

Query: What is the capital of France?
Classification: {'query': 'What is the capital of France?', 'type': 'knowledge_based', 'confidence': 0.7, 'handler': 'rag_system'}

Query: What time is it now?
Classification: {'query': 'What time is it now?', 'type': 'real_time', 'confidence': 0.9, 'handler': 'function_handler'}

Query: What's the weather in Vietnam?
Classification: {'query': "What's the weather in Vietnam?", 'type': 'real_time', 'confidence': 0.9, 'handler': 'function_handler'}

Query: Calculate the square root of 16
Classification: {'query': 'Calculate the square root of 16', 'type': 'function_call', 'confidence': 0.9, 'handler': 'function_handler'}

Query: Tell me about quantum physics
Classification: {'query': 'Tell me about quantum physics', 'type': 'knowledge_based', 'confidence': 0.7, 'handler': 'rag_system'}


In [ ]:
class RAGSystem:
    def __init__(self, vectorstores, llm):
        self.router = QueryRouter()
        self.vector_store = vectorstores  # Your vector store implementation
        self.llm = llm  # Your LLM implementation

    async def process_query(self, query: str):
        # Get routing information
        route = self.router.route_query(query)

        if route["handler"] == "direct_llm":
            return await self.handle_conversational(query)

        elif route["handler"] == "function_handler":
            return await self.handle_function_call(query)

        elif route["handler"] == "rag_system":
            return await self.handle_rag_query(query)

        return "I'm not sure how to handle this query."

    async def handle_conversational(self, query: str):
        # Direct LLM interaction without retrieval
        return await self.llm.generate_response(query)

    async def handle_function_call(self, query: str):
        # Handle real-time queries and function calls
        # Implement your function calling logic here
        pass

    async def handle_rag_query(self, query: str):
        # Traditional RAG pipeline
        docs = await self.vector_store.similarity_search(query)
        context = self.prepare_context(docs)
        return await self.llm.generate_response(query, context)